#### More about this task can be found on the link below:
https://www.kaggle.com/competitions/idc-410-document-classification?rvi=1

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Separate the features and labels in the training data
train_text = train_data['text']
train_labels = train_data['label']

# Split the training data into training and validation sets
train_text, val_text, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.2, random_state=42)

# Print the shape of the training, validation, and test sets
print('Training data shape:', train_text.shape)
print('Validation data shape:', val_text.shape)
print('Test data shape:', test_data.shape)



Training data shape: (960,)
Validation data shape: (240,)
Test data shape: (300, 3)


In [7]:
# Initialize the tokenizer with the maximum number of words to keep
MAX_NUM_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(train_text)

# Convert the text data to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_text)
val_sequences = tokenizer.texts_to_sequences(val_text)
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# Get the maximum sequence length in the training data
MAX_SEQUENCE_LENGTH = max(len(seq) for seq in train_sequences)

# Pad the sequences to the maximum length
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
val_data = pad_sequences(val_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Print the shape of the preprocessed training, validation, and test data
print('Preprocessed training data shape:', train_data.shape)
print('Preprocessed validation data shape:', val_data.shape)
print('Preprocessed test data shape:', test_data.shape)


Preprocessed training data shape: (960, 2364)
Preprocessed validation data shape: (240, 2364)
Preprocessed test data shape: (300, 2364)


In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the train and test data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Tokenize the text data
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 200
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df['text'].values)
train_sequences = tokenizer.texts_to_sequences(train_df['text'].values)
test_sequences = tokenizer.texts_to_sequences(test_df['text'].values)
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Convert the label data to integers
label_to_int = {'sport': 0, 'business': 1, 'politics': 2, 'tech': 3, 'entertainment': 4}
train_df['label'] = train_df['label'].map(label_to_int)
train_labels = train_df['label'].values

# Split the data into training and validation sets
val_size = 0.1
val_samples = int(len(train_data) * val_size)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=val_size)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(MAX_NUM_WORDS, 128, input_length=MAX_SEQUENCE_LENGTH),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),  
      
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
EPOCHS = 35
BATCH_SIZE = 32
history = model.fit(train_data, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_data, val_labels))

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_data, val_labels)
print('Validation accuracy:', val_acc)

# Make predictions on the test set
test_predictions = model.predict(test_data)
test_predictions = np.argmax(test_predictions, axis=1)

# Map the predicted labels back to their text values
int_to_label = {0: 'sport', 1: 'business', 2: 'politics', 3: 'tech', 4: 'entertainment'}
test_predictions = [int_to_label[i] for i in test_predictions]

# Save the predictions to a CSV file
output_df = pd.DataFrame({'index': test_df['index'], 'label': test_predictions})
output_df.to_csv('test_predictions15.csv', index=False)


Epoch 1/35
34/34 [==============================] - 3s 53ms/step - loss: 1.5999 - accuracy: 0.2907 - val_loss: 1.5842 - val_accuracy: 0.2917
Epoch 2/35
34/34 [==============================] - 2s 47ms/step - loss: 1.4130 - accuracy: 0.7630 - val_loss: 1.3548 - val_accuracy: 0.6750
Epoch 3/35
34/34 [==============================] - 1s 44ms/step - loss: 0.8381 - accuracy: 0.8963 - val_loss: 0.7425 - val_accuracy: 0.8333
Epoch 4/35
34/34 [==============================] - 1s 43ms/step - loss: 0.2321 - accuracy: 0.9722 - val_loss: 0.5325 - val_accuracy: 0.8583
Epoch 5/35
34/34 [==============================] - 1s 43ms/step - loss: 0.0560 - accuracy: 0.9907 - val_loss: 0.4885 - val_accuracy: 0.8750
Epoch 6/35
34/34 [==============================] - 1s 43ms/step - loss: 0.0280 - accuracy: 0.9963 - val_loss: 0.4726 - val_accuracy: 0.9083
Epoch 7/35
34/34 [==============================] - 2s 47ms/step - loss: 0.0236 - accuracy: 0.9954 - val_loss: 0.4960 - val_accuracy: 0.9000
Epoch 8/35
34